In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install datasets==2.15

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.0
    Uninstalling fsspec-2024.3.0:
      Successfully uninstalled fsspec-2024.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    F

In [2]:
pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install scikit-multilearn
!pip install datasets

In [4]:
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)


2024-03-25 10:25:03.869166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 10:25:03.869279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 10:25:04.005494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import pandas as pd

In [6]:
sentences_df = pd.read_csv('/kaggle/input/dl-original/training/sentences.tsv', sep='\t')
labels_df = pd.read_csv('/kaggle/input/dl-original/training/labels.tsv', sep='\t')

In [7]:
df1 = sentences_df
df2 = labels_df

In [8]:
import pandas as pd
merged_df = pd.merge(df1, df2, on=['Text-ID', 'Sentence-ID'])
merged_df


,Text-ID,Sentence-ID,Text,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,BG_002,1,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BG_002,2,КВ.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BG_002,3,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BG_002,4,Вече месеци наред той е обект на проверка за к...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG_002,5,Там от години прокуратурата се явява съдружник...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,TR_M_022,12,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,0.0,0.0,0.0,0.0,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,TR_M_022,13,"Vergi tabana yayılacak, yoksul kesimin vergi y...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,TR_M_022,14,Sosyal refahı artıracak destekleyici düzenleme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,TR_M_022,15,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
merged_df = merged_df[merged_df['Text-ID'].str.contains('EN')]

In [10]:
merged_df = merged_df[~merged_df.isin(['0.5', '', pd.NA]).any(axis=1)]
merged_df = merged_df.drop(columns=merged_df.columns[:2])

In [11]:
merged_df = merged_df[~(merged_df == 0.5).any(axis=1)]

In [12]:
X = merged_df.iloc[:, :1]  # Assuming the first two columns are features
Y = merged_df.iloc[:, 1:]  # Assuming the rest of the columns are labels


In [13]:
# Assuming Y is your DataFrame
for column in Y.columns:
    unique_values = Y[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print()


Unique values in column 'Self-direction: thought attained':
[0. 1.]

Unique values in column 'Self-direction: thought constrained':
[0. 1.]

Unique values in column 'Self-direction: action attained':
[0. 1.]

Unique values in column 'Self-direction: action constrained':
[0. 1.]

Unique values in column 'Stimulation attained':
[0. 1.]

Unique values in column 'Stimulation constrained':
[0. 1.]

Unique values in column 'Hedonism attained':
[0. 1.]

Unique values in column 'Hedonism constrained':
[0. 1.]

Unique values in column 'Achievement attained':
[0. 1.]

Unique values in column 'Achievement constrained':
[0. 1.]

Unique values in column 'Power: dominance attained':
[0. 1.]

Unique values in column 'Power: dominance constrained':
[0. 1.]

Unique values in column 'Power: resources attained':
[0. 1.]

Unique values in column 'Power: resources constrained':
[0. 1.]

Unique values in column 'Face attained':
[0. 1.]

Unique values in column 'Face constrained':
[0. 1.]

Unique values in c

In [14]:
import pandas as pd
combined_data = pd.DataFrame({
    'Text': X['Text'],
    'labels': Y.apply(list, axis=1)
})
combined_data

,Text,labels
12765,Hispanic Voters Are Losing Faith In The Democr...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12766,The support of Hispanic voters at the midterms...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12767,U.S. President Joe Biden speaks to employees a...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12768,(Julie Bennett/Getty Images) According to a Qu...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12769,This marks the lowest approval rating of any d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
19004,This will reduce America's vulnerability to en...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19005,Direct Provision was only ever intended to be ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19011,Legislate to provide Irish born children norma...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19013,Extend the ombudsman’s jurisdiction to the app...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
import numpy as np
import random
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict

# Set random seed
random.seed(0)

# Shuffle data
combined_data = combined_data.sample(frac=1, random_state=0).reset_index(drop=True)

# Extract 'Text' and 'labels' columns
text = combined_data['Text'].tolist()
labels = combined_data['labels'].tolist()

# Create label weights
labels = np.array(labels)
label_weights = 1 - labels.sum(axis=0) / labels.sum()

# Stratified train-test split for multilabel dataset
row_ids = np.arange(len(labels))
train_idx, y_train, val_idx, y_val = iterative_train_test_split(row_ids[:, np.newaxis], labels, test_size=0.1)
x_train = [text[i] for i in train_idx.flatten()]
x_val = [text[i] for i in val_idx.flatten()]

# Create Hugging Face dataset
ds = DatasetDict({
    'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
    'val': Dataset.from_dict({'text': x_val, 'labels': y_val})
})


In [16]:
# model name
model_name = 'mistralai/Mistral-7B-v0.1'

# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

# qunatization config
# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # enable 4-bit quantization
    bnb_4bit_quant_type='nf4',  # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant=True,  # quantize quantized weights //insert xzibit meme
    #bnb_4bit_compute_dtype=torch.bfloat16,  # optimized fp format for ML
    
)


# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

labels_array = np.array(labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=labels_array.shape[1]  # Access the shape attribute of labels_array
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Map:   0%|          | 0/5562 [00:00<?, ? examples/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

# define which metrics to compute for evaluation
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average = 'micro')
    f1_macro = f1_score(labels, predictions > 0, average = 'macro')
    f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }


In [18]:
# create custom trainer class to be able to pass label weights and calculate mutilabel loss
class CustomTrainer(Trainer):

    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # compute custom loss
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss


In [19]:
import warnings
import functools
import torch

# Suppress the deprecation warning
warnings.filterwarnings("ignore", message="In the future `np.object` will be defined as the corresponding NumPy scalar.")

# Define training args
training_args = TrainingArguments(
    output_dir='multilabel_classification',
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

# Train
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['val'],
    tokenizer=tokenizer,
    data_collator=functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    label_weights=torch.tensor(label_weights, device=model.device)
)

trainer.train()

# Save model
peft_model_id = 'multilabel_mistral'
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted
1,0.099700,0.038262,0.040201,0.009840,0.035639
2,0.037100,0.031166,0.173160,0.057331,0.137506
3,0.019900,0.034563,0.309677,0.189530,0.269213
4,0.009400,0.042407,0.271186,0.138620,0.251159
5,0.003700,0.050969,0.293907,0.155873,0.275073


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant

('multilabel_mistral/tokenizer_config.json',
 'multilabel_mistral/special_tokens_map.json',
 'multilabel_mistral/tokenizer.model',
 'multilabel_mistral/added_tokens.json',
 'multilabel_mistral/tokenizer.json')

In [ ]:
# sentences_df = pd.read_csv('/kaggle/input/dl-original/validation/sentences.tsv', sep='\t')
# labels_df = pd.read_csv('/kaggle/input/dl-original/validation/labels.tsv', sep='\t')
# df1 = sentences_df
# df2 = labels_df
# import pandas as pd
# merged_df = pd.merge(df1, df2, on=['Text-ID', 'Sentence-ID'])
# merged_df = merged_df[merged_df['Text-ID'].str.contains('EN')]
# merged_df = merged_df[~merged_df.isin(['0.5', '', pd.NA]).any(axis=1)]
# merged_df = merged_df.drop(columns=merged_df.columns[:2])
# merged_df = merged_df[~(merged_df == 0.5).any(axis=1)]
# X = merged_df.iloc[:, :1]  # Assuming the first two columns are features
# Y = merged_df.iloc[:, 1:]  # Assuming the rest of the columns are labels
# import pandas as pd
# combined_data = pd.DataFrame({
#     'Text': X['Text'],
#     'labels': Y.apply(list, axis=1)
# })

In [46]:
from datasets import Dataset

test_text = combined_data['Text'].tolist()
test_labels = combined_data['labels'].tolist()

# Create Hugging Face test dataset
test_dataset = Dataset.from_dict({'text': test_text, 'labels': test_labels})

# Tokenize the test examples
tokenized_test_dataset = test_dataset.map(
    lambda example: tokenizer(example['text']),
    batched=True,
)

# Set the format to 'torch'
tokenized_test_dataset.set_format('torch')

# Test the model
predictions = trainer.predict(tokenized_test_dataset)





Map:   0%|          | 0/6181 [00:00<?, ? examples/s]

AttributeError: 'numpy.ndarray' object has no attribute 'int'

In [49]:
predictions.predictions

array([[ -8.450628 ,  -8.149525 ,  -1.4913938, ...,  -9.419984 ,
         -6.2623763,  -5.943224 ],
       [ -8.868511 , -10.012003 ,  -6.248975 , ...,  -9.882385 ,
         -8.968958 ,  -8.20541  ],
       [ -7.0834837,  -6.1419477,  -5.6137886, ...,  -9.381815 ,
         -5.359112 ,  -6.5315905],
       ...,
       [ -5.7685604, -11.083858 ,  -4.7825737, ..., -12.263563 ,
         -8.066369 ,  -9.830065 ],
       [ -6.5054493,  -6.152956 ,  -2.2597399, ...,  -7.725224 ,
         -5.1426134,  -3.0596461],
       [ -7.9736423,  -7.6082916,  -3.651897 , ...,  -8.719881 ,
         -5.507146 ,  -6.513278 ]], dtype=float32)

In [50]:
# Convert logits to predicted labels
predicted_labels = (predictions.predictions > 0).astype(np.int32).tolist()


# Add predicted labels to the DataFrame
combined_data['Predicted_Labels'] = predicted_labels

# Display the DataFrame with predicted labels
print(combined_data.head())

                                                Text  \
0                                  End the mandates!   
1  However, the DHS expert stressed that emerging...   
2  Ms Ní Sheaghdha told Mary Wilson on this eveni...   
3  The men’s team has agreed to a pay cut to ensu...   
4  Barr was serving as attorney general at the time.   

                                              labels  \
0  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                    Predicted_Labels  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..

In [69]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np

# Assuming true_labels and predicted_labels are DataFrame columns
true_labels = combined_data['labels']
predicted_labels = combined_data['Predicted_Labels']
TL = []
PL = []
for i in true_labels:
    sub = []
    for j in i:
        sub.append(j)
    TL.append(sub)
for i in predicted_labels:
    sub = []
    for j in i:
        sub.append(j)
    PL.append(sub)



label_namesS = []
for i in range(0,38):
  label_namesS.append(str(i))
classification_rep = classification_report(TL,PL, target_names=label_namesS)
print(classification_rep)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        12
           2       1.00      0.03      0.06       103
           3       0.00      0.00      0.00        60
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00         4
           6       1.00      0.11      0.20        18
           7       0.00      0.00      0.00         2
           8       0.71      0.62      0.66       174
           9       1.00      0.14      0.25        92
          10       0.00      0.00      0.00        84
          11       0.00      0.00      0.00        18
          12       1.00      0.07      0.13        70
          13       0.00      0.00      0.00        25
          14       0.00      0.00      0.00        15
          15       1.00      0.06      0.12        31
          16       1.00      0.06      0.12        31
          17       0.00    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np

# Assuming true_labels and predicted_labels are DataFrame columns
true_labels = combined_data['labels']
predicted_labels = combined_data['Predicted_Labels']
TL = []
PL = []
for i in true_labels:
    sub = []
    for j in range(0,len(i),2):
        a = i[j]
        b = i[j+1]

        if a + b >= 0.5 :

            sub.append(1)

        else :
            sub.append(0)
    TL.append(sub)
    
for i in predicted_labels:
    sub = []
    for j in range(0,len(i),2):
        a = i[j]
        b = i[j+1]

        if a + b >= 0.5 :

            sub.append(1)

        else :
            sub.append(0)
    PL.append(sub)



label_namesS = []
for i in range(0,19):
  label_namesS.append(str(i))
classification_rep = classification_report(TL,PL, target_names=label_namesS)
print(classification_rep)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       1.00      0.02      0.04       163
           2       0.00      0.00      0.00        26
           3       1.00      0.10      0.18        20
           4       0.73      0.46      0.56       266
           5       0.00      0.00      0.00       102
           6       1.00      0.05      0.10        95
           7       1.00      0.04      0.08        46
           8       1.00      0.02      0.05        83
           9       0.82      0.42      0.56       332
          10       1.00      0.20      0.34        49
          11       1.00      0.11      0.20       231
          12       1.00      0.03      0.06       100
          13       0.00      0.00      0.00        26
          14       1.00      0.03      0.05        39
          15       0.00      0.00      0.00        39
          16       1.00      0.21      0.34       199
          17       1.00    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
sentences_df = pd.read_csv('/kaggle/input/dl-original/validation/sentences.tsv', sep='\t')
labels_df = pd.read_csv('/kaggle/input/dl-original/validation/labels.tsv', sep='\t')
df1 = sentences_df
df2 = labels_df
import pandas as pd
merged_df = pd.merge(df1, df2, on=['Text-ID', 'Sentence-ID'])
merged_df = merged_df[merged_df['Text-ID'].str.contains('EN')]
merged_df = merged_df[~merged_df.isin(['0.5', '', pd.NA]).any(axis=1)]
merged_df = merged_df.drop(columns=merged_df.columns[:2])
merged_df = merged_df[~(merged_df == 0.5).any(axis=1)]
X = merged_df.iloc[:, :1]  # Assuming the first two columns are features
Y = merged_df.iloc[:, 1:]  # Assuming the rest of the columns are labels
import pandas as pd
combined_data1 = pd.DataFrame({
    'Text': X['Text'],
    'labels': Y.apply(list, axis=1)
})
from datasets import Dataset

test_text = combined_data1['Text'].tolist()
test_labels = combined_data1['labels'].tolist()

# Create Hugging Face test dataset
test_dataset = Dataset.from_dict({'text': test_text, 'labels': test_labels})

# Tokenize the test examples
tokenized_test_dataset = test_dataset.map(
    lambda example: tokenizer(example['text']),
    batched=True,
)

# Set the format to 'torch'
tokenized_test_dataset.set_format('torch')

# Test the model
predictions = trainer.predict(tokenized_test_dataset)

# Convert logits to predicted labels
predicted_labels = (predictions.predictions > 0).astype(np.int32).tolist()


# Add predicted labels to the DataFrame
combined_data1['Predicted_Labels'] = predicted_labels


from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np

# Assuming true_labels and predicted_labels are DataFrame columns
true_labels = combined_data1['labels']
predicted_labels = combined_data1['Predicted_Labels']
TL = []
PL = []
for i in true_labels:
    sub = []
    for j in i:
        sub.append(j)
    TL.append(sub)
for i in predicted_labels:
    sub = []
    for j in i:
        sub.append(j)
    PL.append(sub)



label_namesS = []
for i in range(0,38):
  label_namesS.append(str(i))
classification_rep = classification_report(TL,PL, target_names=label_namesS)
print(classification_rep)

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np

# Assuming true_labels and predicted_labels are DataFrame columns
true_labels = combined_data1['labels']
predicted_labels = combined_data1['Predicted_Labels']
TL = []
PL = []
for i in true_labels:
    sub = []
    for j in range(0,len(i),2):
        a = i[j]
        b = i[j+1]

        if a + b >= 0.5 :

            sub.append(1)

        else :
            sub.append(0)
    TL.append(sub)
    
for i in predicted_labels:
    sub = []
    for j in range(0,len(i),2):
        a = i[j]
        b = i[j+1]

        if a + b >= 0.5 :

            sub.append(1)

        else :
            sub.append(0)
    PL.append(sub)



label_namesS = []
for i in range(0,19):
  label_namesS.append(str(i))
classification_rep = classification_report(TL,PL, target_names=label_namesS)
print(classification_rep)


Map:   0%|          | 0/2042 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00        30
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00         3
           8       0.47      0.47      0.47        70
           9       0.00      0.00      0.00        26
          10       0.00      0.00      0.00        27
          11       0.00      0.00      0.00        11
          12       0.00      0.00      0.00        22
          13       0.00      0.00      0.00        16
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00        10
          16       0.00      0.00      0.00        10
          17       0.00    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.